In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define skill synonyms for normalization
skill_synonyms = {
'nlp': 'natural language processing','ml': 'machine learning','ai': 'artificial intelligence','angular': 'angular js','c programming':'c','responsive web design':'web design','api':'apis',
'js': 'javascript','mvc': 'model view  controller','sql': 'mysql','db': 'database','microservices':'Microservices architecture','rest': 'restful api','responsive design':'web design',
'ux': 'user experience','ui': 'user interface','python programming': 'python','java programming': 'java','c# programming': 'c#','c programming language':'c',
'c++ programming': 'c++','node': 'node.js','aws': 'amazon web services','azure': 'microsoft azure','gcp': 'google cloud platform','cascading style sheets (css)': 'css',
'docker': 'containerization','k8s': 'kubernetes','devops': 'development operations','ci/cd': 'continuous integration/continuous deployment', 'sql server': 'microsoft sql server',
'postgres': 'postgresql','mongodb': 'mongo db','rest': 'restful services','graphql': 'graph query language','flask': 'flask framework','django': 'django framework','spring': 'spring framework',
'express': 'express.js','vue': 'vue.js','jquery': 'j query','sass': 'sassy css','less': 'leaner css','babel': 'babel.js',
'jira software': 'jira','kanban': 'kanban methodology','scrum': 'scrum methodology','agile': 'agile methodology','waterfall': 'waterfall methodology','git': 'git version control','svn': 'subversion',
'bash': 'bash scripting','shell': 'shell scripting','powershell': 'power shell','terraform': 'terraform automation','ansible': 'ansible automation','chef': 'chef automation','puppet': 'puppet automation',
'jenkins': 'jenkins ci/cd','travis': 'travis ci','circleci': 'circle ci','vagrant': 'vagrant automation','linux operating system': 'linux','unix operating system': 'unix','windows operating system': 'windows',
'macos': 'mac operating system','ios': 'ios development','android': 'android development','react native': 'react native development','flutter development': 'flutter','kotlin': 'kotlin programming','swift programming': 'swift',
'objective-c': 'objective-c programming','php programming': 'php','ruby programming': 'ruby','rails': 'ruby on rails','go programming': 'go','rust programming': 'rust','perl programming': 'perl','lua programming': 'lua',
'r': 'r programming','matlab': 'matlab programming','sas programming': 'sas','stata': 'stata programming','hadoop': 'apache hadoop','spark': 'apache spark','hive': 'apache hive','kafka': 'apache kafka','zookeeper': 'apache zookeeper',
'cassandra': 'apache cassandra','hbase': 'apache hbase','storm': 'apache storm','flink': 'apache flink','beam': 'apache beam','pandas library': 'pandas','numpy library': 'numpy','scipy library': 'scipy','scikit-learn library': 'scikit-learn',
'tensorflow library': 'tensorflow','keras library': 'keras','pytorch library': 'pytorch','mxnet library': 'mxnet','xgboost library': 'xgboost','lightgbm library': 'lightgbm','catboost library': 'catboost','opencv library': 'opencv',
'spacy library': 'spacy','nltk library': 'nltk','gensim library': 'gensim','beautifulsoup': 'beautiful soup','selenium': 'selenium automation','webdriver': 'webdriver automation','jupyter': 'jupyter notebook','colab': 'google colab',
'matplotlib': 'matplotlib library','seaborn library': 'seaborn','plotly library': 'plotly','dash library': 'dash','bokeh library': 'bokeh','d3.js library': 'd3.js','tableau': 'tableau software','powerbi': 'microsoft power bi','qlik software': 'qlik',
'looker software': 'looker','superset': 'apache superset','redash': 'redash software','grafana': 'grafana software','splunk': 'splunk software','elk': 'elasticsearch logstash kibana','new relic': 'new relic software','datadog software': 'datadog',
'prometheus software': 'prometheus','nagios software': 'nagios','zabbix software': 'zabbix','pagerduty software': 'pagerduty','slack software': 'slack','teams': 'microsoft teams','zoom software': 'zoom','webex': 'cisco webex',
'bluejeans software': 'bluejeans','skype software': 'skype','trello software': 'trello','asana software': 'asana','notion software': 'notion','confluence software': 'confluence','airtable software': 'airtable','monday.com software': 'monday.com',
'basecamp software': 'basecamp','clickup software': 'clickup','odoo':'odoo software'
}

In [ ]:
def load_and_preprocess_job_description(xpressjob_path, topjob_path):
    # Load the CSV files into DataFrames
    xpressjob_df = pd.read_csv(xpressjob_path)
    topjob_df = pd.read_csv(topjob_path)

    # Combine the DataFrames
    combined_df = pd.concat([xpressjob_df, topjob_df], ignore_index=True)

    # Check for duplicates based on 'Company' and 'Job Title' and drop them
    combined_df.drop_duplicates(subset=['Company', 'Job Title'], inplace=True)

    # Extract the relevant columns
    required_columns = ['Company', 'Job Title', 'Skills', 'Degree Program']
    combined_df = combined_df[required_columns]

    # # Convert the Skills column from list-like strings to plain strings
    # combined_df['Skills'] = combined_df['Skills'].apply(lambda x: ', '.join(eval(x)))

    return combined_df

In [ ]:
xpressjob_path = '/content/drive/My Drive/cv/final_xpressjob_df_v2.csv'
topjob_path = '/content/drive/My Drive/cv/topjobs_final_df_v2.csv'
job_description = load_and_preprocess_job_description(xpressjob_path,topjob_path)

In [ ]:
job_description

,Company,Job Title,Skills,Degree Program
0,Orfium Sri Lanka,Technical Support Services Manager,"Remote Desktop Support, Jira",Bachelor's Degree in Computer Science
1,Tea Tang (Pvt) Ltd,Business Analyst,"SaaS, ERP, APIs",Bachelor's degree in Business Administration
2,Tea Tang (Pvt) Ltd,Software Developer,"Odoo, Full Stack Development, Python, JavaScri...","Bachelor Degree in Computer Science, Bachelor ..."
3,MyWEB information systems,Web Designer and Technical Support,"Web hosting, Web design, Website layout, Wiref...",NaN
4,Emerging Media,Motion Graphics Designer,"Adobe Effect, Premiere Pro, Cinema 4D, Blender",NaN
...,...,...,...,...
525,Expolanka Freight (Pvt) Ltd,Junior Systems Executive,Enterprise Application Access Control Manageme...,"Computer Science or related degree, Higher Dip..."
527,CyberLabs,Consulting Manager,"Cybersecurity, Information Technology","Bachelor's degree in Cybersecurity, Bachelor's..."
528,Mahindra Ideal Finance Limited,Information Security Officer,Certified Information System Security Professi...,"Bachelor's degree in Information Security, Bac..."
529,eNerds Pty Ltd,Desk Engineer,"Microsoft Active Directory Administration, Use...",NaN


In [ ]:
co = len(job_description['Job Title'].unique())
co

322

In [ ]:
job_description['Job Title'].unique()

array(['Technical Support Services Manager', 'Business Analyst',
       'Software Developer', 'Web Designer and Technical Support',
       'Motion Graphics Designer', 'Associate Sofftware Engineer',
       'Business Development Executives', 'SEO Specialist',
       'IT Operations Assistant', 'Deputy Manager',
       'Help Desk Technician', 'Lecturer', 'ERP Support Executive',
       'Enterprise Sales Executive', 'IT', 'IT Technician',
       'Senior Software Architect', 'Senior Designer', 'Manager',
       'Junior Executive', 'Digital Marketing Executive',
       'Senior Test Engineer', 'Sales Executive', 'UI UX Engineer',
       'Sales Administrator', 'Help Desk Technician Intern', 'IT Manager',
       'Senior IT Executive', 'Senior Data Engineer',
       'Senior Quality Assurance Automation Engineer',
       'Senior Software Engineer', 'Developer',
       'Senior Quality Assurance', 'Assistant Manager', 'Web Publisher',
       'Senior Lead Data Engineer', 'Frontend Developer',
      

In [ ]:
degree_synonyms = {
    'bsc computer science': 'bachelor of science in computer science',
    'bs in computer science': 'bachelor of science in computer science',
    'bsc (hons) computer science': 'bachelor of science in computer science',
    'bsc business administration': 'bachelor of business administration',
    'ba (hons) business management': 'bachelor of business administration',
    'bachelor degree in computer science': 'bachelor of computer science',
    'bachelor\'s degree in computer science': 'bachelor of computer science',
    'bachelor\'s degree in business administration': 'bachelor of business administration',
    'bachelor degree in information technology': 'bachelor of information technology',
    'bachelor of information technology': 'bachelor of information technology',
    'bsc (hons) information technology': 'bachelor of science in information technology',
    'bsc in information technology': 'bachelor of science in information technology',
    'bsc cybersecurity technology': 'bachelor of science in cybersecurity technology',
    'bs in cybersecurity technology': 'bachelor of science in cybersecurity technology',
    'bsc (hons) cybersecurity technology': 'bachelor of science in cybersecurity technology',
    'information technology': 'bachelor of information technology',
    'bachelor\'s degree in ict': 'bachelor of information technology',
    'engineering': 'bachelor of engineering',
    'textile management': 'bachelor of textile management',
    'data analytics': 'bachelor of data science',
    'graphic design': 'bachelor of fine arts in graphic design',
    'physical science' : 'bachelor of physical science',
    'business management': 'bachelor of business administration',
    'bs in physical science': 'bachelor of science in physical science',
    'bsc (hons) physical science': 'bachelor of science in physical science',
    'data science': 'bachelor of data science',
    'computer science': 'bachelor of science in computer science',
    'bsc marketing': 'bachelor of science in marketing',
    'bs marketing': 'bachelor of science in marketing',
    'bsc (hons) marketing': 'bachelor of science in marketing',
    'bsc cybersecurity management and policy': 'bachelor of science in cybersecurity management and policy',
    'bs in cybersecurity management and policy': 'bachelor of science in cybersecurity management and policy',
    'bsc (hons) cybersecurity management and policy': 'bachelor of science in cybersecurity management and policy',
    'ba in liberal studies completion program': 'bachelor of arts in liberal studies',
    'ba (hons) liberal studies': 'bachelor of arts in liberal studies',
    'bachelor of science (honours) in data science & artificial intelligence': 'bachelor of science in data science & artificial intelligence',
    'bsc data science & artificial intelligence': 'bachelor of science in data science & artificial intelligence',
    'bsc (hons) data science & artificial intelligence': 'bachelor of science in data science & artificial intelligence',
    'bachelor of applied arts and sciences': 'bachelor of applied arts and sciences',
    'bsc (hons) applied arts and sciences': 'bachelor of applied arts and sciences',
    'bs in applied arts and sciences': 'bachelor of applied arts and sciences',
    'bsc general business': 'bachelor of science in general business',
    'bs general business': 'bachelor of science in general business',
    'bsc (hons) general business': 'bachelor of science in general business',
    'bsc (hons) accounting and finance': 'bachelor of science in accounting and finance',
    'bsc accounting and finance': 'bachelor of science in accounting and finance',
    'bs in accounting and finance': 'bachelor of science in accounting and finance',
    'bsc (hons) project management': 'bachelor of science in project management',
    'bs in project management': 'bachelor of science in project management',
    'bs machine learning': 'bachelor of science in machine learning',
    'bsc (hons) machine learning': 'bachelor of science in machine learning',
    'master in machine learning': 'master of science in machine learning',
    'msc machine learning': 'master of science in machine learning',
    'msc in artificial intelligence': 'master of science in artificial intelligence',
    'master of data science': 'master of science in data science',
    'bsc graphic design': 'bachelor of science in graphic design',
    'bs in graphic design': 'bachelor of science in graphic design',
    'bsc (hons) graphic design': 'bachelor of science in graphic design',
    'bachelor degree electrical engineering': 'bachelor of electrical engineering',
    'bsc electrical engineering': 'bachelor of science in electrical engineering',
    'bs in electrical engineering': 'bachelor of science in electrical engineering',
    'bsc (hons) electrical engineering': 'bachelor of science in electrical engineering',
    'bachelor of fine arts in graphic design': 'bachelor of fine arts in graphic design',
    'bfa in graphic design': 'bachelor of fine arts in graphic design',
    'ba in graphic design': 'bachelor of fine arts in graphic design',
    'bachelor of business administration': 'bachelor of business administration',
    'bba': 'bachelor of business administration',
    'bachelor\'s degree in business analysis' : 'bachelor of business analysis',
    'bachelor degree in business administration': 'bachelor of business administration',
    'bachelor degree in business studies': 'bachelor of business administration',
    'electrical engineering': 'bachelor of electrical engineering',
    'mechanical engineering': 'bachelor of mechanical engineering',
    'graphic web designer': 'bachelor of fine arts in graphic design',
    'graphic designing': 'bachelor of fine arts in graphic design'
}

In [ ]:
job_description

,Company,Job Title,Skills,Degree Program
0,Orfium Sri Lanka,Technical Support Services Manager,"Remote Desktop Support, Jira",Bachelor's Degree in Computer Science
1,Tea Tang (Pvt) Ltd,Business Analyst,"SaaS, ERP, APIs",Bachelor's degree in Business Administration
2,Tea Tang (Pvt) Ltd,Software Developer,"Odoo, Full Stack Development, Python, JavaScri...","Bachelor Degree in Computer Science, Bachelor ..."
3,MyWEB information systems,Web Designer and Technical Support,"Web hosting, Web design, Website layout, Wiref...",NaN
4,Emerging Media,Motion Graphics Designer,"Adobe Effect, Premiere Pro, Cinema 4D, Blender",NaN
...,...,...,...,...
525,Expolanka Freight (Pvt) Ltd,Junior Systems Executive,Enterprise Application Access Control Manageme...,"Computer Science or related degree, Higher Dip..."
527,CyberLabs,Consulting Manager,"Cybersecurity, Information Technology","Bachelor's degree in Cybersecurity, Bachelor's..."
528,Mahindra Ideal Finance Limited,Information Security Officer,Certified Information System Security Professi...,"Bachelor's degree in Information Security, Bac..."
529,eNerds Pty Ltd,Desk Engineer,"Microsoft Active Directory Administration, Use...",NaN


In [ ]:
degree_df=pd.read_csv('/content/drive/My Drive/cv/degree_data_with_clusters_v5.csv')
degree_with_cluster=pd.read_csv('/content/drive/My Drive/cv/clustered_degree_df_v5.csv')

In [ ]:
degree_with_cluster

,degree,related_clusters
0,bachelor of science in computer science,3
1,bachelor of business administration,2
2,bachelor of information technology,1
3,bachelor of science in cybersecurity technology,7
4,bachelor of science in marketing,11
...,...,...
173,doctorate of business administration in busine...,2
174,doctor of philosophy in computer science,3
175,online doctor of philosophy in computer science,3
176,phd in computer and information science,3


In [ ]:
degree_df

,category,degree_name,University,application_link,cluster
0,Computer Science,bachelor of science in computer science,University of London,https://www.coursera.org/degrees/bachelor-of-s...,3
1,Business Administration,bachelor of business administration,University of London,https://www.coursera.org/degrees/bsc-business-...,2
2,Information Technology,bachelor of information technology,Illinois Institute of Technology,https://www.coursera.org/degrees/bach-informat...,1
3,Cybersecurity,bachelor of science in cybersecurity technology,University of Maryland Global Campus,https://www.coursera.org/degrees/bachelor-scie...,7
4,Marketing,bachelor of science in marketing,University of London,https://www.coursera.org/degrees/bsc-marketing...,11
...,...,...,...,...,...
326,Computer Science,phd in computer science,"University of Maryland, College Park",https://www.cs.umd.edu/,3
327,Computer Science,phd in computer science,"University of California, Berkeley",https://www2.eecs.berkeley.edu/,3
328,Computer Science,phd in computer science,University of Massachusetts Amherst,https://www.cics.umass.edu/,3
329,Computer Science,phd in computer science & engineering,University of Washington,https://www.cs.washington.edu/,3


In [ ]:
!pip install anvil-uplink
!pip install gensim

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [ ]:
import anvil.server
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api
from difflib import get_close_matches


In [ ]:
# Load pre-trained GloVe embeddings
glove_model = api.load("glove-wiki-gigaword-300")

In [ ]:
def get_embedding(skill, embeddings=glove_model, embedding_dim=300):
  words = skill.split()  # Split the skill string into individual words
  clean_words = [word.strip() for word in words]  # Remove extra spaces from each word
  vectors = [embeddings[word] for word in clean_words if word in embeddings]
  if vectors:
    return np.mean(vectors, axis=0).reshape(1, -1)  # Return the mean embedding vector
  else:
    return None

In [ ]:
def recommend_courses(skill_gap, processed_data, cluster_skills_df, max_recommendations_per_skill=3):
    recommended_courses = {}

    for missing_skill in skill_gap:
        recommended_courses[missing_skill] = []
        seen_courses = set()  # Track seen courses to avoid duplicates
        course_coverage = {}  # Track skills covered by each course

        normalized_missing_skill = skill_synonyms.get(missing_skill, missing_skill)
        print(f"Normalized missing skill: {normalized_missing_skill}")

        # Find related clusters for the missing skill
        clusters = cluster_skills_df[cluster_skills_df['skill'] == normalized_missing_skill]['related_clusters']

        if clusters.empty:
            print(f"No clusters found for skill: {normalized_missing_skill}")
            continue

        cluster_labels = clusters.values[0].split(', ')
        print(f"Cluster labels for skill '{normalized_missing_skill}': {cluster_labels}")

        for cluster_label in cluster_labels:
            cluster_label = int(cluster_label)

            # Get courses in the current cluster
            relevant_courses = processed_data[processed_data['cluster_labels'] == cluster_label]
            print(f"Found {len(relevant_courses)} courses in cluster {cluster_label} for skill '{normalized_missing_skill}'")

            for _, course in relevant_courses.iterrows():
                course_skills_list = course['skills']

                if isinstance(course_skills_list, str):
                    course_skills_list = re.split(r',|\[|\]|\{|\}', course_skills_list)
                course_skills_list = [skill.strip().lower() for skill in course_skills_list if skill.strip()]
                print(f"Course skills list: {course_skills_list}")

                # Try to calculate similarity using embeddings
                try:
                    missing_skill_embedding = get_embedding(normalized_missing_skill)

                    if missing_skill_embedding is None:
                        raise ValueError(f"Embedding for missing skill '{normalized_missing_skill}' is None")

                    highest_similarity = 0
                    best_cover_skill = None
                    embedding_based_match = False

                    for cover_skill in course_skills_list:
                        cover_skill_embedding = get_embedding(cover_skill)

                        if cover_skill_embedding is None:
                            print(f"Embedding for cover skill '{cover_skill}' is None")
                            continue  # Skip if cover skill embedding is None

                        similarity = cosine_similarity(missing_skill_embedding, cover_skill_embedding)[0][0]
                        if similarity > highest_similarity:
                            highest_similarity = similarity
                            best_cover_skill = cover_skill
                            embedding_based_match = True

                    # Recommend the course if similarity is above the threshold
                    if embedding_based_match and highest_similarity >= 0.5:
                        course_name = course['course']
                        if course_name not in seen_courses:
                            recommended_courses[missing_skill].append({
                                'course_name': course_name,
                                'skills_covered': course['skills'],
                                'rating': course['rating'],
                                'highest_similarity': highest_similarity,
                                'best_cover_skill': best_cover_skill
                            })
                            seen_courses.add(course_name)

                except Exception as e:
                    print(f"Error with embedding, falling back to alternative method for '{normalized_missing_skill}' and course '{course['course']}': {e}")

                    # Fallback: Check if the normalized missing skill is in the course skills list
                    for cover_skill in course_skills_list:
                        if normalized_missing_skill in cover_skill or cover_skill in normalized_missing_skill:
                            course_name = course['course']
                            if course_name not in seen_courses:  # Check if the course is already added
                                recommended_courses[missing_skill].append({
                                    'course_name': course_name,
                                    'skills_covered': course['skills'],
                                    'rating': course['rating'],
                                    'best_cover_skill': cover_skill
                                })
                                seen_courses.add(course_name)  # Mark the course as seen
                                course_coverage[course_name] = {
                                    'course_name': course_name,
                                    'skills_covered': course['skills'],
                                    'rating': course['rating'],
                                    'best_cover_skills': set([cover_skill])
                                }
                            else:
                                course_coverage[course_name]['best_cover_skills'].add(cover_skill)

            # Sort recommended courses by similarity first, then by rating
            recommended_courses[missing_skill] = sorted(
                recommended_courses[missing_skill],
                key=lambda x: (x.get('highest_similarity', 0), x['rating']),
                reverse=True
            )[:max_recommendations_per_skill]

    # Update recommended_courses to include aggregated best_cover_skills
    for skill in recommended_courses:
        for course in recommended_courses[skill]:
            course_name = course['course_name']
            if course_name in course_coverage:
                course['best_cover_skills'] = list(course_coverage[course_name]['best_cover_skills'])

    return recommended_courses


# def recommend_courses(skill_gap, processed_data, cluster_skills_df, max_recommendations_per_skill=3):
#     recommended_courses = {}

#     for missing_skill in skill_gap:
#         recommended_courses[missing_skill] = []

#         normalized_missing_skill = skill_synonyms.get(missing_skill, missing_skill)
#         print(f"Normalized missing skill: {normalized_missing_skill}")

#         # Find related clusters for the missing skill
#         clusters = cluster_skills_df[cluster_skills_df['skill'] == normalized_missing_skill]['related_clusters']

#         if clusters.empty:
#             print(f"No clusters found for skill: {normalized_missing_skill}")
#             continue

#         cluster_labels = clusters.values[0].split(', ')
#         print(f"Cluster labels for skill '{normalized_missing_skill}': {cluster_labels}")

#         for cluster_label in cluster_labels:
#             cluster_label = int(cluster_label)

#             # Get courses in the current cluster
#             relevant_courses = processed_data[processed_data['cluster_labels'] == cluster_label]
#             print(f"Found {len(relevant_courses)} courses in cluster {cluster_label} for skill '{normalized_missing_skill}'")

#             for _, course in relevant_courses.iterrows():
#                 course_skills_list = course['skills']

#                 if isinstance(course_skills_list, str):
#                     course_skills_list = re.split(r',|\[|\]|\{|\}', course_skills_list)
#                 course_skills_list = [skill.strip().lower() for skill in course_skills_list if skill.strip()]
#                 print(f"Course skills list: {course_skills_list}")

#                 # Check if the course covers the normalized missing skill
#                 try:
#                     missing_skill_embedding = get_embedding(normalized_missing_skill)

#                     if missing_skill_embedding is None:
#                         print(f"Embedding for missing skill '{normalized_missing_skill}' is None")
#                         continue  # Skip if missing skill embedding is None

#                     highest_similarity = 0
#                     best_cover_skill = None

#                     for cover_skill in course_skills_list:
#                         cover_skill_embedding = get_embedding(cover_skill)

#                         if cover_skill_embedding is None:
#                             print(f"Embedding for cover skill '{cover_skill}' is None")
#                             continue  # Skip if cover skill embedding is None

#                         similarity = cosine_similarity(missing_skill_embedding, cover_skill_embedding)[0][0]
#                         if similarity >= 0.7:
#                             print(f"Comparing '{normalized_missing_skill}' with '{cover_skill}', similarity: {similarity}")

#                         if similarity > highest_similarity:
#                             highest_similarity = similarity
#                             best_cover_skill = cover_skill

#                     # After checking all cover skills, print the highest similarity and best cover skill
#                     if highest_similarity >= 0.5:  # Adjust similarity threshold as needed
#                         recommended_courses[missing_skill].append({
#                             'course_name': course['course'],
#                             'skills_covered': course['skills'],
#                             'rating': course['rating'],
#                             'highest_similarity': highest_similarity,
#                             'best_cover_skill': best_cover_skill
#                         })
#                         print(f"Skill: {missing_skill}, Course: {course['course']}, Highest Similarity: {highest_similarity}, Best Cover Skill: {best_cover_skill}")

#                 except Exception as e:
#                     print(f"Error processing '{missing_skill}' and course '{course['course']}': {e}")

#             # Sort recommended courses by similarity first, then by rating
#             recommended_courses[missing_skill] = sorted(
#                 recommended_courses[missing_skill],
#                 key=lambda x: (x['highest_similarity'], x['rating']),
#                 reverse=True
#             )[:max_recommendations_per_skill]

#     return recommended_courses



# working with one skill gap

# def recommend_courses(skill_gap, processed_data, cluster_skills_df, max_recommendations_per_skill=3):
#     recommended_courses = {}

#     for missing_skill in skill_gap:
#         recommended_courses[missing_skill] = []
#         seen_courses = set()  # Track seen courses to avoid duplicates

#         normalized_missing_skill = skill_synonyms.get(missing_skill, missing_skill)
#         print(f"Normalized missing skill: {normalized_missing_skill}")

#         # Find related clusters for the missing skill
#         clusters = cluster_skills_df[cluster_skills_df['skill'] == normalized_missing_skill]['related_clusters']

#         if clusters.empty:
#             print(f"No clusters found for skill: {normalized_missing_skill}")
#             continue

#         cluster_labels = clusters.values[0].split(', ')
#         print(f"Cluster labels for skill '{normalized_missing_skill}': {cluster_labels}")

#         for cluster_label in cluster_labels:
#             cluster_label = int(cluster_label)

#             # Get courses in the current cluster
#             relevant_courses = processed_data[processed_data['cluster_labels'] == cluster_label]
#             print(f"Found {len(relevant_courses)} courses in cluster {cluster_label} for skill '{normalized_missing_skill}'")

#             for _, course in relevant_courses.iterrows():
#                 course_skills_list = course['skills']

#                 if isinstance(course_skills_list, str):
#                     course_skills_list = re.split(r',|\[|\]|\{|\}', course_skills_list)
#                 course_skills_list = [skill.strip().lower() for skill in course_skills_list if skill.strip()]
#                 print(f"Course skills list: {course_skills_list}")

#                 # Check if the course covers the normalized missing skill
#                 best_cover_skill = None

#                 for cover_skill in course_skills_list:
#                     if normalized_missing_skill in cover_skill or cover_skill in normalized_missing_skill:
#                         best_cover_skill = cover_skill
#                         break

#                 if best_cover_skill:
#                     course_name = course['course']
#                     if course_name not in seen_courses:  # Check if the course is already added
#                         recommended_courses[missing_skill].append({
#                             'course_name': course_name,
#                             'skills_covered': course['skills'],
#                             'rating': course['rating'],
#                             'best_cover_skill': best_cover_skill
#                         })
#                         seen_courses.add(course_name)  # Mark the course as seen
#                         print(f"Skill: {missing_skill}, Course: {course_name}, Best Cover Skill: {best_cover_skill}")

#             # Sort recommended courses by rating
#             recommended_courses[missing_skill] = sorted(recommended_courses[missing_skill], key=lambda x: x['rating'], reverse=True)[:max_recommendations_per_skill]

#     return recommended_courses

# def recommend_courses(skill_gap, processed_data, cluster_skills_df, max_recommendations_per_skill=3):
#     recommended_courses = {}

#     for missing_skill in skill_gap:
#         recommended_courses[missing_skill] = []
#         seen_courses = set()  # Track seen courses to avoid duplicates
#         course_coverage = {}  # Track skills covered by each course

#         normalized_missing_skill = skill_synonyms.get(missing_skill, missing_skill)
#         print(f"Normalized missing skill: {normalized_missing_skill}")

#         # Find related clusters for the missing skill
#         clusters = cluster_skills_df[cluster_skills_df['skill'] == normalized_missing_skill]['related_clusters']

#         if clusters.empty:
#             print(f"No clusters found for skill: {normalized_missing_skill}")
#             continue

#         cluster_labels = clusters.values[0].split(', ')
#         print(f"Cluster labels for skill '{normalized_missing_skill}': {cluster_labels}")

#         for cluster_label in cluster_labels:
#             cluster_label = int(cluster_label)

#             # Get courses in the current cluster
#             relevant_courses = processed_data[processed_data['cluster_labels'] == cluster_label]
#             print(f"Found {len(relevant_courses)} courses in cluster {cluster_label} for skill '{normalized_missing_skill}'")

#             for _, course in relevant_courses.iterrows():
#                 course_skills_list = course['skills']

#                 if isinstance(course_skills_list, str):
#                     course_skills_list = re.split(r',|\[|\]|\{|\}', course_skills_list)
#                 course_skills_list = [skill.strip().lower() for skill in course_skills_list if skill.strip()]
#                 print(f"Course skills list: {course_skills_list}")

#                 # Check if the course covers the normalized missing skill
#                 for cover_skill in course_skills_list:
#                     if normalized_missing_skill in cover_skill or cover_skill in normalized_missing_skill:
#                         course_name = course['course']
#                         if course_name not in seen_courses:  # Check if the course is already added
#                             recommended_courses[missing_skill].append({
#                                 'course_name': course_name,
#                                 'skills_covered': course['skills'],
#                                 'rating': course['rating'],
#                                 'best_cover_skill': cover_skill
#                             })
#                             seen_courses.add(course_name)  # Mark the course as seen
#                             course_coverage[course_name] = {
#                                 'course_name': course_name,
#                                 'skills_covered': course['skills'],
#                                 'rating': course['rating'],
#                                 'best_cover_skills': set([cover_skill])
#                             }
#                         else:
#                             course_coverage[course_name]['best_cover_skills'].add(cover_skill)

#             # Sort recommended courses by rating
#             recommended_courses[missing_skill] = sorted(recommended_courses[missing_skill], key=lambda x: x['rating'], reverse=True)[:max_recommendations_per_skill]

#     # Update recommended_courses to include aggregated best_cover_skills
#     for skill in recommended_courses:
#         for course in recommended_courses[skill]:
#             course_name = course['course_name']
#             if course_name in course_coverage:
#                 course['best_cover_skills'] = list(course_coverage[course_name]['best_cover_skills'])

#     return recommended_courses

In [ ]:
# def identify_skill_gap(required_skills, user_skills):
#     # Normalize and split required and user skills
#     required_skills_set = {skill.strip().lower() for skill in required_skills.split(',')}
#     user_skills_set = {skill.strip().lower() for skill in user_skills.split(',')}

#     skill_gap = []

#     for req_skill in required_skills_set:
#         req_skill_normalized = skill_synonyms.get(req_skill, req_skill)
#         req_embedding = get_embedding(req_skill_normalized)

#         found_match = False

#         for user_skill in user_skills_set:
#             user_skill_normalized = skill_synonyms.get(user_skill, user_skill)
#             user_embedding = get_embedding(user_skill_normalized)

#             if req_embedding is None or user_embedding is None:
#                 print(f"Skipping comparison as one of the embeddings is None for '{req_skill_normalized}' or '{user_skill_normalized}'")
#                 continue

#             try:
#                 similarity = cosine_similarity(req_embedding, user_embedding)[0][0]
#                 print(f"Comparing '{req_skill_normalized}' with '{user_skill_normalized}', similarity: {similarity}")

#                 if similarity >= 0.7:  # Using the threshold
#                     found_match = True
#                     break

#             except Exception as e:
#                 print(f"Error processing skills '{req_skill_normalized}' and '{user_skill_normalized}': {e}")

#         if not found_match:
#             skill_gap.append(req_skill_normalized)
#             print(f"'{req_skill_normalized}' is identified as missing based on similarity threshold.")

#     return skill_gap

# def identify_skill_gap(required_skills, user_skills):
#     # Normalize and split required and user skills
#     required_skills_set = {skill.strip().lower() for skill in required_skills.split(',')}
#     user_skills_set = {skill.strip().lower() for skill in user_skills.split(',')}

#     skill_gap = []

#     for req_skill in required_skills_set:
#         req_skill_normalized = skill_synonyms.get(req_skill, req_skill)
#         req_embedding = get_embedding(req_skill_normalized)

#         if req_embedding is None:
#             print(f"Skipping '{req_skill_normalized}' as its embedding is None.")
#             continue

#         found_match = False

#         for user_skill in user_skills_set:
#             user_skill_normalized = skill_synonyms.get(user_skill, user_skill)
#             user_embedding = get_embedding(user_skill_normalized)

#             if user_embedding is None:
#                 print(f"Skipping '{user_skill_normalized}' as its embedding is None.")
#                 continue

#             try:
#                 similarity = cosine_similarity(req_embedding, user_embedding)[0][0]
#                 print(f"Comparing '{req_skill_normalized}' with '{user_skill_normalized}', similarity: {similarity}")

#                 if similarity >= 0.7:
#                     found_match = True
#                     break

#             except Exception as e:
#                 print(f"Error processing skills '{req_skill_normalized}' and '{user_skill_normalized}': {e}")

#         if not found_match:
#             skill_gap.append(req_skill_normalized)
#             print(f"'{req_skill_normalized}' is identified as missing based on similarity threshold.")

#     return skill_gap

def identify_skill_gap(required_skills, user_skills):
    # Normalize and split required and user skills
    required_skills_set = {skill.strip().lower() for skill in required_skills.split(',')}
    user_skills_set = {skill.strip().lower() for skill in user_skills.split(',')}

    skill_gap = []

    for req_skill in required_skills_set:
        req_skill_normalized = skill_synonyms.get(req_skill, req_skill)
        req_embedding = get_embedding(req_skill_normalized)

        if req_embedding is None:
            print(f"Embedding for required skill '{req_skill_normalized}' is None. Falling back to string matching.")
        else:
            print(f"Embedding for required skill '{req_skill_normalized}' found.")

        found_match = False

        for user_skill in user_skills_set:
            user_skill_normalized = skill_synonyms.get(user_skill, user_skill)
            user_embedding = get_embedding(user_skill_normalized)

            if user_embedding is None:
                print(f"Embedding for user skill '{user_skill_normalized}' is None. Falling back to string matching.")
            else:
                print(f"Embedding for user skill '{user_skill_normalized}' found.")

            try:
                if req_embedding is not None and user_embedding is not None:
                    # Use embeddings for similarity check
                    similarity = cosine_similarity(req_embedding, user_embedding)[0][0]
                    print(f"Comparing '{req_skill_normalized}' with '{user_skill_normalized}', similarity: {similarity}")

                    if similarity >= 0.7:
                        found_match = True
                        break
                else:
                    # Fallback to string matching if embeddings are None
                    if req_skill_normalized in user_skill_normalized or user_skill_normalized in req_skill_normalized:
                        found_match = True
                        print(f"String match found between '{req_skill_normalized}' and '{user_skill_normalized}'.")
                        break

            except Exception as e:
                print(f"Error processing skills '{req_skill_normalized}' and '{user_skill_normalized}': {e}")

        if not found_match:
            skill_gap.append(req_skill_normalized)
            print(f"'{req_skill_normalized}' is identified as missing based on similarity threshold or string match.")

    return skill_gap


In [ ]:
def check_job_vacancy(company_name, job_title):
    # Mapping between short terms and full job titles
    job_title_mapping = {
        'se': 'Software Engineer',
        'ase': 'Associate Software Engineer',
        'tl': 'Team Lead',
        'pm': 'Project Manager',
        'qa': 'Quality Assurance Engineer',
        'ba': 'Business Analyst',
        'at': 'Technical Analyst',
        'sse': 'Senior Software Engineer',
        'atl': 'Associate Team Lead',
        'uiux': 'UI/UX Engineer',
        'hr': 'Human Resource Manager',
        'devops': 'DevOps Engineer',
        'ml': 'Machine Learning Engineer',
        'dl': 'Deep Learning Engineer',
        'ai': 'Artificial Intelligence Engineer',
        'fe dev': 'Front End Developer',
        'be dev': 'Back End Developer',
        'fs dev': 'Full Stack Developer',
        'data sci': 'Data Scientist',
        'data an': 'Data Analyst',
        'data eng': 'Data Engineer',
        'sre': 'Site Reliability Engineer',
        'cloud eng': 'Cloud Engineer',
        'sysadmin': 'System Administrator',
        'dba': 'Database Administrator',
        'cto': 'Chief Technology Officer',
        'cfo': 'Chief Financial Officer',
        'cio': 'Chief Information Officer',
        'it tech': 'IT Technician'
    }

    # Convert company name and job title to lowercase
    company_name_lower = company_name.lower().strip()
    job_title_lower = job_title.lower().strip()

    # Check if job title is a short term and map it to the full job title
    job_title_full = job_title_mapping.get(job_title_lower, job_title_lower)

    # Check if DataFrame contains required columns
    if not all(col in job_description.columns for col in ['Company', 'Job Title', 'Skills', 'Degree Program']):
        raise ValueError("DataFrame must contain 'Company', 'Job Title', 'Skills', and 'Degree Program' columns.")

    # Fuzzy matching for company names
    possible_companies = job_description['Company'].str.lower().unique()
    close_company_matches = get_close_matches(company_name_lower, possible_companies)
    print(f"Possible company matches found: {close_company_matches}")

    if close_company_matches:
        closest_company = close_company_matches[0]
        print(f"Closest company match found: {closest_company}")

        # Filter DataFrame for rows with matching company name and job title (case-insensitive)
        matching_company_rows = job_description['Company'].str.lower().str.contains(closest_company, regex=False)
        matching_rows = matching_company_rows & job_description['Job Title'].str.lower().str.contains(job_title_full.lower(), regex=False)

        if matching_rows.any():
            matched_data = job_description[matching_rows]
            job_skills = matched_data['Skills'].iloc[0]
            job_degree = matched_data['Degree Program'].iloc[0]
            print(f"Returning skills: {job_skills}, degree: {job_degree}")
            return closest_company, job_title_full, job_skills, job_degree
        else:
            # Fuzzy matching for job titles if no exact match
            possible_titles = job_description['Job Title'].str.lower().unique()
            close_title_matches = get_close_matches(job_title_full.lower(), possible_titles)
            print(f"Possible title matches found: {close_title_matches}")
            if close_title_matches:
                closest_title = close_title_matches[0]
                print(f"Did you mean: {close_title_matches}? Closest title: {closest_title}")

                matching_rows_fuzzy = matching_company_rows & job_description['Job Title'].str.lower().str.contains(closest_title, regex=False)
                if matching_rows_fuzzy.any():
                    matched_data_fuzzy = job_description[matching_rows_fuzzy]
                    job_skills = matched_data_fuzzy['Skills'].iloc[0]
                    job_degree = matched_data_fuzzy['Degree Program'].iloc[0]
                    print(f"Returning skills: {job_skills}, degree: {job_degree}")
                    return closest_company, closest_title, job_skills, job_degree

    # If no matches found, return None
    return None, None, None, None

# def check_job_vacancy(company_name, job_title):
#   # Mapping between short terms and full job titles
#   job_title_mapping = {
#       'se': 'Software Engineer',
#       'ase': 'Associate Software Engineer',
#       'tl': 'Team Lead',
#       'pm': 'Project Manager',
#       'qa': 'Quality Assurance Engineer',
#       'ba': 'Business Analyst',
#       'at': 'Technical Analyst',
#       'sse': 'Senior Software Engineer',
#       'atl': 'Associate Team Lead',
#       'uiux': 'UI/UX Engineer',
#       'hr': 'Human Resource Manager',
#       'devops': 'DevOps Engineer',
#       'ml': 'Machine Learning Engineer',
#       'dl': 'Deep Learning Engineer',
#       'ai': 'Artificial Intelligence Engineer',
#       'fe dev': 'Front End Developer',
#       'be dev': 'Back End Developer',
#       'fs dev': 'Full Stack Developer',
#       'data sci': 'Data Scientist',
#       'data an': 'Data Analyst',
#       'data eng': 'Data Engineer',
#       'sre': 'Site Reliability Engineer',
#       'cloud eng': 'Cloud Engineer',
#       'sysadmin': 'System Administrator',
#       'dba': 'Database Administrator',
#       'cto': 'Chief Technology Officer',
#       'cfo': 'Chief Financial Officer',
#       'cio': 'Chief Information Officer',
#       'it tech': 'IT Technician'
#   }

#   # Convert company name and job title to lowercase
#   company_name_lower = company_name.lower().strip()
#   job_title_lower = job_title.lower().strip()

#   # Debug print statements
#   print(f"Company Name (Lowercase): {company_name_lower}")
#   print(f"Job Title (Lowercase): {job_title_lower}")

#   # Check if job title is a short term and map it to the full job title
#   if job_title_lower in job_title_mapping:
#       job_title_full = job_title_mapping[job_title_lower]
#   else:
#       job_title_full = job_title_lower

#   # Debug print statement
#   print(f"Mapped Job Title: {job_title_full}")

#   # Check if company name exists (case-insensitive)
#   if job_description['Company'].str.lower().str.contains(company_name_lower, regex=False).any():
#       # Filter DataFrame for rows with matching company name (case-insensitive)
#       matching_company_rows = job_description['Company'].str.lower().str.contains(company_name_lower, regex=False)
#       # Check if any row matches the company name and contains the job title (case-insensitive)
#       matching_rows = matching_company_rows & job_description['Job Title'].str.lower().str.contains(job_title_full.lower(), regex=False)

#       # Check if any row matches both company and job title
#       if matching_rows.any():
#           # Extract the required skills for the matching job
#           job_skills = job_description.loc[matching_rows, 'Skills'].iloc[0]
#           job_degree = job_description.loc[matching_rows, 'Degree Program'].iloc[0]
#           print(f"Returning skills: {job_skills}, degree: {job_degree}")  # Debug print
#           return job_skills, job_degree

#   # If no matches found, return None
#   return None, None

In [ ]:
updated_df=pd.read_csv('/content/drive/My Drive/cv/final_hierarchical_clusted_df_v5.csv')
cover_skills_df=pd.read_csv('/content/drive/My Drive/cv/final_hie_cover_skills_v5.csv')

In [ ]:
updated_df

,course,skills,rating,x_scaled,y_scaled,cluster_labels
0,Google Cybersecurity,"network security,python,linux,cloud computing,...",4.800000,1.076593,-1.621520,324
1,Google Data Analytics,"data analysis,r programming,mysql,business com...",4.800000,1.623089,-0.869816,444
2,Google Project Management:,"project management,strategy and operations,lea...",4.800000,2.115618,-0.828596,435
3,Google Digital Marketing & E-commerce,"digital marketing,marketing,marketing manageme...",4.800000,1.467667,-0.228347,418
4,Google IT Support,"computer networking,network architecture,netwo...",4.800000,1.464740,-0.866553,446
...,...,...,...,...,...,...
4317,Introduction to SQL,"sql subqueries,sql joins,sql window functions,...",4.660532,-0.346058,-0.934006,317
4318,Experimental Design & Recommendations,"smart experiments,interpreting test results,ex...",4.660532,1.223982,-0.977388,430
4319,Become a business intelligence analyst.,basic understanding of how to read different t...,4.660532,2.250134,-0.214041,414
4320,Learn multi-cloud computing.,"cloud solution evaluation,cloud governance str...",4.660532,0.134949,-0.276678,362


In [ ]:
cover_skills_df

,skill,related_clusters
0,network security,"385, 129, 387, 4, 261, 262, 135, 136, 131, 396..."
1,python,"1, 4, 6, 10, 15, 16, 17, 18, 19, 22, 24, 25, 2..."
2,linux,"1, 26, 32, 34, 41, 43, 50, 53, 54, 61, 93, 94,..."
3,cloud computing,"1, 4, 6, 14, 15, 17, 19, 20, 22, 23, 27, 28, 2..."
4,algorithms,"1, 5, 6, 7, 10, 11, 12, 15, 16, 18, 21, 22, 23..."
...,...,...
5913,cloud architecture proficiency,362
5914,ai fairness,245
5915,ai governance,245
5916,ai transparency,245


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# @anvil.server.callable
# def handle_submission(job_title, company_name, skills, degree):
#     try:
#         print(f"Received data:\nJob Title: {job_title}\nCompany Name: {company_name}\nSkills: {skills}\nDegree: {degree}")

#         required_skills,required_degree = check_job_vacancy(company_name, job_title)
#         if required_skills:
#             print("Job vacancy exists for the specified job title in the company.")
#             skill_gap = identify_skill_gap(required_skills, skills)

#             if skill_gap:
#                 recommended_courses = recommend_courses(skill_gap, updated_df, cover_skills_df, max_recommendations_per_skill=3)

#                 result = {
#                     "status": f"Job vacancy exists for the specified '{job_title}' in the '{company_name}'.",
#                     "required_skills": required_skills,
#                     "user_skills": skills,
#                     "missing_skills": skill_gap,
#                     "recommended_courses": recommended_courses
#                 }
#             else:
#                 result = f"You have all the required skills for the job '{job_title}' at '{company_name}'. You are fit for the role!"
#         else:
#             result = f"The company '{company_name}' does not have a job vacancy for the job title '{job_title}'. Please try another company."

#         print(result)
#         return result

#     except Exception as e:
#         error_msg = f"Error processing submission for job title '{job_title}' and company '{company_name}': {str(e)}"
#         print(error_msg)
#         return {"error": error_msg}

@anvil.server.callable
def handle_submission(job_title, company_name, skills, user_degree):
    try:
        print(f"Received data:\nJob Title: {job_title}\nCompany Name: {company_name}\nSkills: {skills}\nDegree: {user_degree}")

        # Check and retrieve required skills and degree
        company, title, required_skills, required_degree = check_job_vacancy(company_name, job_title)

        if not company or not title or not required_skills:
            return {
                "status": f"No job vacancy found for the specified job title '{job_title}' and company '{company_name}'.",
                "required_skills": "None",
                "required_degree": "None"
            }

        print(f"Required Skills: {required_skills}, Required Degree: {required_degree}")

        # Initialize result dictionary
        result = {}

        # Validate types
        if not isinstance(required_skills, (list, str)):
            raise ValueError("Expected list or string for required_skills.")
        if not isinstance(required_degree, (list, str)):
            required_degree = []

        if required_skills:
            print("Job vacancy exists for the specified job title in the company.")

            # Handle missing or empty skills input
            if not skills or skills.strip() == '':
                skills = '-'
            elif not isinstance(skills, (list, str)):
                raise ValueError("Expected list or string for skills.")

            # Identify skill gaps
            skill_gap = identify_skill_gap(required_skills, skills)
            print(f"Skill Gap: {skill_gap}")

            # Recommend courses based on the skill gap
            if skill_gap:
                recommended_courses = recommend_courses(skill_gap, updated_df, cover_skills_df, max_recommendations_per_skill=3)
                print(f"Recommended Courses: {recommended_courses}")

                result.update({
                    "status": f"Job vacancy exists for the specified '{title}' in the '{company}'.",
                    "required_skills": required_skills,
                    "user_skills": skills,
                    "missing_skills": skill_gap,
                    "recommended_courses": recommended_courses if recommended_courses else "No recommendations available."
                })
            else:
                result.update({
                    "status": f"You have all the required skills for the job '{title}' at '{company}'. You are fit for the role!",
                    "required_skills": required_skills,
                    "user_skills": skills,
                    "missing_skills": "None",
                    "recommended_courses": "No recommendations available."
                })

            # Add degree information if required
            if required_degree:
                result["required_degree"] = required_degree

                if user_degree.lower() not in [rd.lower() for rd in required_degree]:
                    # Find the best suitable degree from the degree_df
                    suitable_degrees = find_suitable_degree(required_degree, user_degree, degree_with_cluster, degree_df)
                    print(f"Suitable Degrees: {suitable_degrees}")

                    result["recommended_degrees"] = suitable_degrees if suitable_degrees else "No recommendations available."
                else:
                    result["recommended_degrees"] = "Your degree matches the required degrees with sufficient similarity."

                if not user_degree or user_degree.strip() == '':
                    user_degree = '-'
                result["user_degree"] = user_degree

            else:
                result["recommended_degrees"] = "No specific degree requirement mentioned."

        else:
            result["status"] = f"The company '{company_name}' does not have a job vacancy for the job title '{job_title}'. Please try another company."

        print(result)
        return result

    except Exception as e:
        error_msg = f"Error processing submission for job title '{job_title}' and company '{company_name}': {str(e)}"
        print(error_msg)
        return {"error": error_msg}




In [ ]:
anvil.server.connect("server_KS6CNKY6GZ6ILPDJLF52QQ32-XQGNCH2RUG2VEQAC")

In [ ]:
# Keep the notebook running to listen for incoming calls
anvil.server.wait_forever()

Streaming output truncated to the last 5000 lines.
Course skills list: ['marketing', 'strategy', 'strategy and operations', 'customer analysis', 'sales', 'product strategy', 'business analysis', 'leadership and management', 'market analysis', 'product marketing', 'design and product', 'product management', 'entrepreneurship', 'market research', 'accounting', 'decision making', 'game theory', 'b2b sales', 'cost accounting', 'data analysis', 'product lifecycle', 'marketing management', 'retail sales']
Course skills list: ['computer programming', 'python', 'programming principles', 'mathematics', 'algorithms', 'computational logic', 'computational thinking', 'computer programming tools', 'critical thinking', 'problem solving', 'theoretical computer science', 'visualization (computer graphics)', 'computer graphics', 'data structures', 'graph theory', 'probability & statistics', 'combinatorics']
Course skills list: ['continuous delivery', 'inventory management', 'leadership and management',

In [ ]:
def find_suitable_degree(required_degrees, user_degrees, clustered_degree_df, degree_data):
    # Helper function to normalize degrees using synonyms
    def normalize_degree(degree, synonyms):
        degree = degree.strip().lower()
        return synonyms.get(degree, degree)

    # Normalize degrees
    required_degrees = [normalize_degree(degree, degree_synonyms) for degree in required_degrees.split(',')]
    user_degrees = [normalize_degree(degree, degree_synonyms) for degree in user_degrees.split(',')]

    print(f"Normalized required degrees: {required_degrees}")
    print(f"Normalized user degrees: {user_degrees}")

    # Identify clusters for required degrees
    required_clusters = []
    for degree in required_degrees:
        cluster_row = clustered_degree_df[clustered_degree_df['degree'].str.lower() == degree]
        if not cluster_row.empty:
            required_clusters.append(cluster_row['related_clusters'].values[0])

    if not required_clusters:
        return {"status": "No cluster found for the required degrees."}

    print(f"Identified clusters for required degrees: {required_clusters}")

    # Filter degrees in degree_data based on identified clusters
    filtered_degree_df = degree_data[degree_data['cluster'].isin(required_clusters)]

    # List of related words to be omitted
    related_terms = {'related field'}

    # Helper function to preprocess degrees to remove related words
    def preprocess_degree(degree, related_terms):
        # Remove related terms from the degree
        words = set(degree.split())
        filtered_words = words - set(related_terms)
        return ' '.join(filtered_words)

    # Preprocess required degrees
    processed_required_degrees = [preprocess_degree(degree, related_terms) for degree in required_degrees]

    # Compute embeddings for processed required degrees and user degrees
    required_embeddings = [get_embedding(degree) for degree in processed_required_degrees]
    user_embeddings = [get_embedding(degree) for degree in user_degrees]

    # Filter out None embeddings
    required_embeddings = [emb for emb in required_embeddings if emb is not None]
    user_embeddings = [emb for emb in user_embeddings if emb is not None]

    # Handle case where user does not provide a degree
    if not user_degrees or not user_embeddings:
        user_emb_match_status = False  # No direct match status check

    # Handle case where no valid embeddings are found
    if not required_embeddings:
        return {"degree_status": "No valid embeddings found for required degrees."}

    # Fuzzy match user degrees with required degrees
    matched_degrees = {}
    for user_degree in user_degrees:
        closest_matches = get_close_matches(user_degree, required_degrees, n=1, cutoff=0.6)
        if closest_matches:
            matched_degrees[user_degree] = closest_matches[0]

    if matched_degrees:
        print(f"Fuzzy matched degrees: {matched_degrees}")

    # Check for direct matches with similarity >= 0.7
    if user_embeddings:
        for req_emb in required_embeddings:
            for user_emb in user_embeddings:
                sim = cosine_similarity(user_emb, req_emb)
                if sim[0][0] >= 0.7:
                    return {"status": "Your degree matches the required degrees with sufficient similarity."}
        user_emb_match_status = True  # Proceed to find recommendations

    # Find suitable degrees if no direct match is found or no user degree provided
    recommended_degrees = []
    for req_degree in processed_required_degrees:
        req_emb = get_embedding(req_degree)
        if req_emb is not None:
            for _, row in filtered_degree_df.iterrows():
                degree_text = row['degree_name']
                degree_embedding = get_embedding(degree_text)

                if degree_embedding is not None:
                    degree_similarity = cosine_similarity(degree_embedding, req_emb)
                    print(f"Degree: {degree_text}, Similarity: {degree_similarity[0][0]}")

                    if degree_similarity[0][0] >= 0.7:
                        degree_info = {
                            'degree_name': row['degree_name'],
                            'university': row['University'],
                            'application_link': row['application_link'],
                            'similarity': degree_similarity[0][0]
                        }
                        recommended_degrees.append(degree_info)

    # Sort by similarity and limit to top 2
    recommended_degrees = sorted(recommended_degrees, key=lambda x: x['similarity'], reverse=True)[:2]

    if not recommended_degrees:
        if user_emb_match_status:
            return {"degree_status": "No suitable degrees found. Consider checking your degree requirements or try different degrees."}
        else:
            return {"degree_status": "No valid embeddings found for degrees."}

    print(f"Recommended degrees: {recommended_degrees}")

    return {"degrees": recommended_degrees}

# def find_suitable_degree(required_degrees, user_degrees, clustered_degree_df, degree_data):
#     # Helper function to normalize degrees using synonyms
#     def normalize_degree(degree, synonyms):
#         degree = degree.strip().lower()
#         return synonyms.get(degree, degree)

#     # Normalize degrees
#     required_degrees = [normalize_degree(degree, degree_synonyms) for degree in required_degrees.split(',')]
#     user_degrees = [normalize_degree(degree, degree_synonyms) for degree in user_degrees.split(',')]

#     print(f"Normalized required degrees: {required_degrees}")
#     print(f"Normalized user degrees: {user_degrees}")

#     # Identify clusters for required degrees
#     required_clusters = []
#     for degree in required_degrees:
#         cluster_row = clustered_degree_df[clustered_degree_df['degree'].str.lower() == degree]
#         if not cluster_row.empty:
#             required_clusters.append(cluster_row['related_clusters'].values[0])

#     if not required_clusters:
#         return {"status": "No cluster found for the required degrees."}

#     print(f"Identified clusters for required degrees: {required_clusters}")

#     # Filter degrees in degree_data_with_clusters based on identified clusters
#     filtered_degree_df = degree_data[degree_data['cluster'].isin(required_clusters)]

#     # List of related words to be omitted
#     related_terms = {'related field'}

#     # Helper function to preprocess degrees to remove related words
#     def preprocess_degree(degree, related_terms):
#         # Remove related terms from the degree
#         words = set(degree.split())
#         filtered_words = words - set(related_terms)
#         return ' '.join(filtered_words)

#     # Preprocess required degrees
#     processed_required_degrees = [preprocess_degree(degree, related_terms) for degree in required_degrees]

#     # Compute embeddings for processed required degrees and user degrees
#     required_embeddings = [get_embedding(degree) for degree in processed_required_degrees]
#     user_embeddings = [get_embedding(degree) for degree in user_degrees]

#     # Filter out None embeddings
#     required_embeddings = [emb for emb in required_embeddings if emb is not None]
#     user_embeddings = [emb for emb in user_embeddings if emb is not None]

#     # Handle case where user does not provide a degree
#     if not user_degrees or not user_embeddings:
#         user_emb_match_status = False  # No direct match status check

#     # Handle case where no valid embeddings are found
#     if not required_embeddings:
#         return {"degree_status": "No valid embeddings found for required degrees."}

#     # Check for direct matches with similarity >= 0.7
#     if user_embeddings:
#         for req_emb in required_embeddings:
#             for user_emb in user_embeddings:
#                 sim = cosine_similarity(user_emb, req_emb)
#                 if sim[0][0] >= 0.7:
#                     return {"status": "Your degree matches the required degrees with sufficient similarity."}
#         user_emb_match_status = True  # Proceed to find recommendations

#     # Find suitable degrees if no direct match is found or no user degree provided
#     recommended_degrees = []
#     for req_degree in processed_required_degrees:
#         req_emb = get_embedding(req_degree)
#         if req_emb is not None:
#             for _, row in filtered_degree_df.iterrows():
#                 degree_text = row['degree_name']
#                 degree_embedding = get_embedding(degree_text)

#                 if degree_embedding is not None:
#                     degree_similarity = cosine_similarity(degree_embedding, req_emb)
#                     print(f"Degree: {degree_text}, Similarity: {degree_similarity[0][0]}")

#                     if degree_similarity[0][0] >= 0.6:
#                         degree_info = {
#                             'degree_name': row['degree_name'],
#                             'university': row['University'],
#                             'application_link': row['application_link'],
#                             'similarity': degree_similarity[0][0]
#                         }
#                         recommended_degrees.append(degree_info)

#     # Sort by similarity and limit to top 2
#     recommended_degrees = sorted(recommended_degrees, key=lambda x: x['similarity'], reverse=True)[:2]

#     if not recommended_degrees:
#         if user_emb_match_status:
#             return {"degree_status": "No suitable degrees found. Consider checking your degree requirements or try different degrees."}
#         else:
#             return {"degree_status": "No valid embeddings found for degrees."}

#     print(f"Recommended degrees: {recommended_degrees}")

#     return {"degrees": recommended_degrees}


In [ ]:
# # working
# def find_suitable_degree(required_degrees, user_degrees, degree_df):
#     # Helper function to normalize degrees using synonyms
#     def normalize_degree(degree, synonyms):
#         degree = degree.strip().lower()
#         return synonyms.get(degree, degree)

#     # Normalize degrees
#     required_degrees = [normalize_degree(degree, degree_synonyms) for degree in required_degrees.split(',')]
#     user_degrees = [normalize_degree(degree, degree_synonyms) for degree in user_degrees.split(',')]

#     print(f"Normalized required degrees: {required_degrees}")
#     print(f"Normalized user degrees: {user_degrees}")

#     # Helper function to preprocess degrees to remove related words
#     def preprocess_degree(degree, related_terms):
#         # Remove related terms from the degree
#         words = set(degree.split())
#         filtered_words = words - set(related_terms)
#         return ' '.join(filtered_words)

#     # List of related words to be omitted
#     related_terms = {'related field'}

#     # Preprocess required degrees
#     processed_required_degrees = [preprocess_degree(degree, related_terms) for degree in required_degrees]

#     # Compute embeddings for processed required degrees and user degrees
#     required_embeddings = [get_embedding(degree) for degree in processed_required_degrees]
#     user_embeddings = [get_embedding(degree) for degree in user_degrees]

#     # Filter out None embeddings
#     required_embeddings = [emb for emb in required_embeddings if emb is not None]
#     user_embeddings = [emb for emb in user_embeddings if emb is not None]

#     # Check if we have valid embeddings
#     if not required_embeddings or not user_embeddings:
#         return {"status": "No valid embeddings found for degrees."}

#     # Check for direct matches with similarity >= 0.7
#     for req_emb in required_embeddings:
#         for user_emb in user_embeddings:
#             sim = cosine_similarity(user_emb, req_emb)
#             if sim[0][0] >= 0.7:
#                 return {"status": "Your degree matches the required degrees with sufficient similarity."}

#     # Find suitable degrees if no direct match is found
#     recommended_degrees = []
#     for req_degree in processed_required_degrees:
#         req_emb = get_embedding(req_degree)
#         if req_emb is not None:
#             for _, row in degree_df.iterrows():
#                 degree_text = row['degree_name']
#                 degree_embedding = get_embedding(degree_text)

#                 if degree_embedding is not None:
#                     degree_similarity = cosine_similarity(degree_embedding, req_emb)
#                     print(f"Degree: {degree_text}, Similarity: {degree_similarity[0][0]}")

#                     if degree_similarity[0][0] >= 0.6:
#                         degree_info = {
#                             'degree_name': row['degree_name'],
#                             'university': row['University'],
#                             'application_link': row['application_link'],
#                             'similarity': degree_similarity[0][0]
#                         }
#                         recommended_degrees.append(degree_info)

#     # Sort by similarity and limit to top 2
#     recommended_degrees = sorted(recommended_degrees, key=lambda x: x['similarity'], reverse=True)[:2]

#     if not recommended_degrees:
#         return {"status": "No suitable degrees found. Consider checking your degree requirements or try different degrees."}

#     print(f"Recommended degrees: {recommended_degrees}")

#     return recommended_degrees

# # def find_suitable_degree(required_degrees, user_degrees, degree_df):
# #     # Helper function to normalize degrees using synonyms
# #     def normalize_degree(degree, synonyms):
# #         degree = degree.strip().lower()
# #         return synonyms.get(degree, degree)

# #     # Normalize degrees
# #     required_degrees = [normalize_degree(degree, degree_synonyms) for degree in required_degrees.split(',')]
# #     user_degrees = [normalize_degree(degree, degree_synonyms) for degree in user_degrees.split(',')]

# #     print(f"Normalized required degrees: {required_degrees}")
# #     print(f"Normalized user degrees: {user_degrees}")

# #     # Helper function to preprocess degrees to remove related words
# #     def preprocess_degree(degree, related_terms):
# #         # Remove related terms from the degree
# #         words = set(degree.split())
# #         filtered_words = words - set(related_terms)
# #         return ' '.join(filtered_words)

# #     # List of related words to be omitted
# #     related_terms = {'related field'}

# #     # Preprocess required degrees
# #     processed_required_degrees = [preprocess_degree(degree, related_terms) for degree in required_degrees]

# #     # Compute embeddings for processed required degrees and user degrees
# #     required_embeddings = [get_embedding(degree) for degree in processed_required_degrees]
# #     user_embeddings = [get_embedding(degree) for degree in user_degrees]

# #     # Filter out None embeddings
# #     required_embeddings = [emb for emb in required_embeddings if emb is not None]
# #     user_embeddings = [emb for emb in user_embeddings if emb is not None]

# #     # Check if we have valid embeddings
# #     if not required_embeddings or not user_embeddings:
# #         return {"status": "No valid embeddings found for degrees."}

# #     # Store recommendations for each required degree
# #     all_recommendations = {}

# #     # Find suitable degrees for each required degree
# #     for req_degree, req_emb in zip(processed_required_degrees, required_embeddings):
# #         if req_emb is not None:
# #             recommendations = []
# #             for _, row in degree_df.iterrows():
# #                 degree_text = row['degree_name']
# #                 degree_embedding = get_embedding(degree_text)

# #                 if degree_embedding is not None:
# #                     degree_similarity = max(cosine_similarity(degree_embedding, user_emb) for user_emb in user_embeddings)
# #                     print(f"Degree: {degree_text}, Similarity: {degree_similarity}")

# #                     if degree_similarity >= 0.6:
# #                         degree_info = row.to_dict()
# #                         degree_info['similarity'] = degree_similarity
# #                         recommendations.append(degree_info)

# #             # Sort recommendations by similarity and limit to top 2
# #             recommendations = sorted(recommendations, key=lambda x: x['similarity'], reverse=True)[:2]
# #             all_recommendations[req_degree] = recommendations

# #     if not all_recommendations:
# #         return {"status": "No suitable degrees found. Consider checking your degree requirements or try different degrees."}

# #     print(f"Recommendations: {all_recommendations}")

# #     return all_recommendations


In [ ]:
test_new_df = degree_with_cluster.copy()

In [ ]:
test_new_df.head()

,degree,related_clusters
0,bachelor of science in computer science,3
1,bachelor of business administration,2
2,bachelor of information technology,1
3,bachelor of science in cybersecurity technology,7
4,bachelor of science in marketing,11


In [ ]:
test_clusters = test_new_df.loc[test_new_df['degree'] == 'bachelor of business administration']['related_clusters']
test_clusters

1    2
Name: related_clusters, dtype: int64

In [ ]:
job_description

,Company,Job Title,Skills,Degree Program
0,Orfium Sri Lanka,Technical Support Services Manager,"Remote Desktop Support, Jira, Zendesk, Confluence",Bachelor's Degree in Computer Science
1,Tea Tang (Pvt) Ltd,Business Analyst - ERP,"SaaS, ERP, APIs",Bachelor's degree in a related field of Busine...
2,Tea Tang (Pvt) Ltd,Software Developer - Odoo,"Odoo, Full Stack Development, Python, JavaScri...","Bachelor Degree in Computer Science, Bachelor ..."
3,MyWEB information systems,Web Designer and Technical Support | Yakkala,"Web hosting, Web design, Website layout, Wiref...",NaN
4,Emerging Media,Motion Graphics Designer,"Adobe Effect, Premiere Pro, Cinema 4D, Blender",NaN
...,...,...,...,...
538,Courtaulds Clothing Lanka (Pvt) Ltd,Senior Systems Engineer (1),"MCSE/MCSA, A+, CCTV Maintenance","Advanced Diploma Degree, Professional Qualific..."
539,CyberLabs,Consulting Manager - Cybersecurity,"Cybersecurity, Information Technology","Bachelor's degree in Cybersecurity, Bachelor's..."
540,Mahindra Ideal Finance Limited,Information Security Officer,Certified Information System Security Professi...,"Bachelor's degree in Information Security, Bac..."
541,eNerds Pty Ltd,Level 1 Service Desk Engineer,"Microsoft Active Directory Administration, Use...",NaN


In [ ]:
df = df.drop(228, axis=0)

In [ ]:
df.to_csv('/content/drive/My Drive/cv/topjobs_final_df_v2.csv', index=False)

In [ ]:
df.loc[307, 'Degree Program'] = 'Information Technology'